In [1]:
import math
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, Flatten
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras.models import load_model
import csv
from keras.layers import Dense, GlobalAveragePooling2D

c:\users\czho9\anaconda3\envs\tsf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Changeable parameter
train_dir = 'D:\capstone\dataset\similar_10calss\FGVC_similar_10class_200_train\CV_0'
val_dir = 'D:\capstone\dataset\similar_10calss\FGVC_similar_10class_200_validation\CV_0'
batch_size = 32
epoch_num = 12
load_prev_model = False
# Make sure to change this to the designated model
# prev_model_path = r'test_mini_4_epoch.h5'
model_metrics_path = r'InceptionV3_similar_10class_200_bottleneck-history.csv'

In [3]:
# Non-changeable parameter
cur_model_path = r'D:\capstone\model_save\InceptionV3_similar_10class_200_bottleneck.h5'

In [4]:
#Limiting the number of resources used, hopefully this works
#It kinda does, CPU usage is only around 40-50% ish
config = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
set_session(tf.Session(config=config))

In [5]:
# Default configuration from
# https://keras.io/preprocessing/image/
# With a little bit of change in parameter
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        val_dir,
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='categorical')

Found 1600 images belonging to 10 classes.
Found 400 images belonging to 10 classes.


In [6]:
if load_prev_model == False:
    # Just to make sure that this is only being loaded once
    base_model = InceptionV3(include_top=False,weights='imagenet',input_shape=(299,299,3))

    x = base_model.output
    # Output shape is (1,1,2048) no need for GlobalAveragePooling
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(train_generator.num_classes, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freezing all the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # and then compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
else:
    model = load_model(prev_model_path)

In [7]:
# TRAINING CELL
num_train_samples = train_generator.samples
train_epoch_steps = math.ceil(num_train_samples / batch_size)
num_val_samples = validation_generator.samples
val_epoch_steps = math.ceil(num_val_samples / batch_size)
train_history = model.fit_generator(train_generator,
                    steps_per_epoch=train_epoch_steps,
                    epochs=epoch_num,
                    validation_data=validation_generator,
                    validation_steps=val_epoch_steps)

Epoch 1/12
50/50 [==============================] - 65s 1s/step - loss: 2.3302 - acc: 0.2056 - val_loss: 1.9354 - val_acc: 0.3500
Epoch 2/12
50/50 [==============================] - 61s 1s/step - loss: 1.8621 - acc: 0.3612 - val_loss: 1.8776 - val_acc: 0.3650
Epoch 3/12
50/50 [==============================] - 57s 1s/step - loss: 1.6504 - acc: 0.4300 - val_loss: 1.8928 - val_acc: 0.3650
Epoch 4/12
50/50 [==============================] - 58s 1s/step - loss: 1.5509 - acc: 0.4569 - val_loss: 1.9747 - val_acc: 0.3800
Epoch 5/12
50/50 [==============================] - 57s 1s/step - loss: 1.4864 - acc: 0.4800 - val_loss: 1.8254 - val_acc: 0.4225
Epoch 6/12
50/50 [==============================] - 58s 1s/step - loss: 1.4141 - acc: 0.5194 - val_loss: 1.7636 - val_acc: 0.4425
Epoch 7/12
50/50 [==============================] - 64s 1s/step - loss: 1.2938 - acc: 0.5550 - val_loss: 1.9855 - val_acc: 0.3675
Epoch 8/12
50/50 [==============================] - 63s 1s/step - loss: 1.2418 - acc: 0.56

In [9]:
# Record the train history in the designated csv file
# Note: newline has to be '' due to the way csv writerow works
with open(model_metrics_path, 'a+', newline='') as history_file:
    csv_writer = csv.writer(history_file)
    for e in train_history.epoch:
        epoch_number = e
        csv_writer.writerow([epoch_number,
                            train_history.history['acc'][e],
                            train_history.history['loss'][e],
                            train_history.history['val_acc'][e],
                            train_history.history['val_loss'][e]])

In [10]:
# Optionally save the model
#model.save(prev_model_path)
model.save(cur_model_path)